In [3]:
# 序文
最近土日に*Scalaでデータ分析を行うにはどうすればよいか* についてまとめており、
ある程度まとめられる段階に粒度にはなってきたので、記事にしていきます。



Adding 11 artifact(s)


In [ ]:
## なぜScalaでデータ分析を行う必要があるのか?
R,Python,Juliaなど、巷にはデータ分析に適したライブラリとパフォーマンスを持つ言語があふれています。
その中で、なぜ、あえてScalaなのかというと、上記の言語はすべて、
*レポート程度の小~中規模の開発には適しているが、大規模開発には適していないからです。*
今後、よりデータ分析が活発化すれば、dAdでScalaを選択したように
大規模開発にも耐えうる言語でのデータ分析の必要性が出てくるのは必須なので、そのための
下準備として、今回Scalaでのデータ分析の形を模索しようと思って記事にしました。
もう一つの理由として、データ分析でも、*やはり重要になるのは、パフォーマンスです。*ですので、データ分析のテクニックから
広告配信に応用できる部分もあるかも知れません。
上記のように、データ分析⇆広告配信という一見すると別分野に思える分野をScalaという言語でつなぐ事が、Scaladでデータ分析を
行う事のおもな理由です。
また、データ分析をPythonで行ってもなじみがなくて、とっつきにくい感が出てしまいますが、Scalaならば少なくとも
TECならば読める方が多いので、データ分析を理解するための一助になるだろうと思ったのも一つの理由です。




In [ ]:
## 環境構築について
分析で必須になる行列計算をScalaでサポートするために、*[Breeze]()*というライブラリを導入しました。
このライブラリにより、行列計算がサポートされると同時に、スライスなど、pythonでおなじみの記法がサポートされているので、
pythonになれた人間にも非常に取っ付きやすく、同時に関数型言語の特性も持っているので、Scalaとpythonの双方のよいところをつかった
計算が可能になります。
エディタについては、レポート単位の小規模な開発の場合は、[Jupyter]() などのレポートを出力できるエディタを扱える便利です。そこで今回のこの記事は
*Jupyerにサードパーティーが開発している[Scalaカーネル]()を*入れて書きました。


In [ ]:
## Scalaによる具体的なメリット
- Option型、Try型など、エラーを出さない仕組みが存在する: データの前処理の段階
- 関数型なので、学習用に適用するロジックを値として外部から注入できる: 学習モデルの形成の段階
- 並列処理が容易なので、交差検証が簡単: 学習の段階

In [1]:
classpath.add("org.scalanlp" %% "breeze" % "0.11.2")

Adding 11 artifact(s)


In [2]:
classpath.add("org.scalanlp" %% "breeze-natives" % "0.11.2")

Adding 17 artifact(s)


In [ ]:
# Breezeによる基礎的な計算と応用

In [ ]:
## ベクトルの簡単な計算

Breezeには、行列計算、代数計算、統計処理などをするためのライブラリが多く含まれている。
まず、数値計算の基礎となるベクトルを扱うために、DenseVectorについて、基礎的な計算方法について書き出していく。

In [5]:
import breeze.linalg._

import breeze.linalg._

In [31]:
// ベクトルを作成
val v= DenseVector(1.0,2.0)

v: DenseVector[Double] = DenseVector(1.0, 2.0)

In [18]:
//要素の 1番目にアクセス
v(0)

res17: Double = 1.0

In [ ]:
pythonと同様にネガティブな値にもキーは対応している

In [9]:
// 最後の要素にアクセス
v(-1)

res8: Double = 2.0

In [8]:
DenseVector型はミュータブルである。
理由は、データ分析では非常に大きな行列式の計算もありうるため、イミュータブルだと、計算をするたびに新しい行列を
生成しなくてはならず、メモリを圧迫するためだからである。

res7: DenseVector[Double] = DenseVector(1.0, 2.0)

In [20]:
v(1)=10.0
v(1)

res19_1: Double = 10.0

In [ ]:
:* 全ての要素に同じ値をかける

In [32]:
val v2=DenseVector(1.0,2.0,3.0)
v:*2.0

7 29, 2017 8:10:55 午後 com.github.fommil.jni.JniLoader liberalLoad
情報: successfully loaded /var/folders/ng/kxk_y29n5xn0jyb_cr7_nc140000gq/T/jniloader8694351817932733089netlib-native_system-osx-x86_64.jnilib


v2: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
res31_1: DenseVector[Double] = DenseVector(2.0, 4.0)

In [ ]:
vはDenseVector[Double]型なので、Int型を掛ける事とエラーが出る
暗黙の型変換があえて行われないことで、数値計算で別のData型に変換される事を防いでいる

In [26]:
v:*2 // エラー

: 

In [ ]:
:+ ベクトル同士を加減する

In [24]:
v
v2
v:+v2

res23_0: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
res23_1: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
res23_2: DenseVector[Double] = DenseVector(2.0, 4.0, 6.0)

In [ ]:
定数の場合、全ての要素に対象の値が加減される

In [27]:
// DenseVector(1.0,2.0):+DenseVector(2.0,2.0)と同じ
v:+2.0

res26: DenseVector[Double] = DenseVector(3.0, 4.0, 5.0)

In [ ]:
要素数が同じベクトル同士でないと、実行時にエラーが出る

In [34]:
val v3=DenseVector(1.0,2.0,3.0)

v.length //2
v3.length //3

// v:+v3

v3: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
res33_1: Int = 2
res33_2: Int = 3

In [ ]:
:= 値を書き換える

In [38]:
val v4=DenseVector(4.0,5.0,6.0)

v4: DenseVector[Double] = DenseVector(4.0, 5.0, 6.0)

In [39]:
v4:+v3

// 値は変わっていない
v4

res38_0: DenseVector[Double] = DenseVector(5.0, 7.0, 9.0)
res38_1: DenseVector[Double] = DenseVector(4.0, 5.0, 6.0)

In [40]:


// 値が入れ替わる
v4:+=v3
v4

res39_0: DenseVector[Double] = DenseVector(5.0, 7.0, 9.0)
res39_1: DenseVector[Double] = DenseVector(5.0, 7.0, 9.0)

In [ ]:
dot 内積を求める

In [36]:
v dot v2

res35: Double = 14.0

In [ ]:
なにがDenseというと、要素がぎっしりつまっているということである。  
Dense(密な)とついているという事は、Sparse(疎な)Vectorも存在する。  
疎なベクトルというのは、たとえば [0,0,0,0,0,1,0,0]のように殆どゼロで構成されているベクトルの事をいう。
例えば、CVした広告のデータなどは、ユーザーがCVする確率がきわめて低いため、非常にスパースなデータである。
殆ど0(=同じ値)という事は、メモリを効率化するチャンスがあるということである。  
そのため、Breezeでは、疎なベクトルのために、SparseVector型とHashVector型という型が用意されている
SparseVectorはメモリ最適化がHashVectorよりもされているが、ゼロでない値に対しての計算がおそい、
対して、HashVectorの方は、Hash値により最適化している分、SparseVectorよりはメモリ効率がよくないが、
計算スピードはSpaseVectorよりも高い。

In [47]:
### ベクトルを生成する
python(正確にはnumpy)のように、さまざまなverctorを生成するメソッドが多く存在する。

vs: SparseVector[Double] = SparseVector((0,2.0), (1,1000.0))
vs2: SparseVector[Double] = SparseVector((0,2.0), (1,5000.0))

In [48]:
// 要素が1の１００次元ベクトルを生成する
val vOnes=DenseVector.ones[Double](100)
vOnes(0)
vOnes.length

vOnes: DenseVector[Double] = DenseVector(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
res47_1: Double = 1.0
res47_2: Int = 100

In [49]:
// 0から1.0まで10等分した10要素のベクトルを生成する
val vLine=linspace(0.0,1.0,10)

vLine: DenseVector[Double] = DenseVector(0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0)

In [51]:
//  関数を引数に与える事により、値を変換する

val vTab=DenseVector.tabulate[Double](4){identity}
val vTab2=DenseVector.tabulate[Double](4){i=>i*5.0}

vTab: DenseVector[Double] = DenseVector(0.0, 1.0, 2.0, 3.0)
vTab2: DenseVector[Double] = DenseVector(0.0, 5.0, 10.0, 15.0)

In [54]:
// Arrayを引数として生成できる
val vAry=DenseVector(Array(1,2,3))


// splat operatorを使用する事で、Seqなどからも生成できる
val vSeq=DenseVector(Seq(1,2):_*)

vAry: DenseVector[Int] = DenseVector(1, 2, 3)
vSeq: DenseVector[Int] = DenseVector(1, 2)

In [56]:
### 要素へのアクセス

vSeq: SparseVector[Int] = SparseVector((0,1), (1,2))

In [64]:
val v100=DenseVector[Double](1,2,3,4,5)

//  一番目から三番目までにアクセス
v(0 to 3)

// 一番目から二番目間でアクセス python でいう[0:3]
v(1 until 3)

// 要素をリバース
v(v.length-1 to  0 by -1 )

v: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0, 5.0)
res63_1: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0)
res63_2: DenseVector[Double] = DenseVector(2.0, 3.0)
res63_3: DenseVector[Double] = DenseVector(5.0, 4.0, 3.0, 2.0, 1.0)

In [ ]:
上記は、新しいインスタンスを生成しているのではなく、元のベクトルと同じポインタにアクセス
している。よって、メモリは上記の処理では全く消費してない

In [79]:
// 値にフィルターをかける
val filter=v(v:<3.0)
// メモリを割り当てる
val filtered=filter.toDenseVector

filter: SliceVector[Int, Double] = breeze.linalg.SliceVector@46b88c34
filtered: DenseVector[Double] = DenseVector(1.0, 2.0)

In [90]:
// 0から10までの要素を持つベクトルの偶数の値だけ0にする
val vEven=linspace(0,1.0,11)
vEven(0 to 10 by 2):=0.0
vEven

vEven: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)
res89_1: DenseVector[Double] = DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
res89_2: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)

In [ ]:
### 行列
当然、行列もBrezeはサポートしている

In [96]:
val m=DenseMatrix((1.0,2.0,3.0),(4.0,5.0,6.0))
val m2=DenseMatrix((1.0,2.0,3.0),(4.0,5.0,6.0))

// 置換
m.t

// 一行目にアクセス
m(0,::)

// 一列目にアクセス
m(::,0)
//  行列積
m*m2.t

m: DenseMatrix[Double] = 1.0  2.0  3.0  
4.0  5.0  6.0  
m2: DenseMatrix[Double] = 1.0  2.0  3.0  
4.0  5.0  6.0  
res95_2: DenseMatrix[Double] = 1.0  4.0  
2.0  5.0  
3.0  6.0  
res95_3: Transpose[DenseVector[Double]] = Transpose(DenseVector(1.0, 2.0, 3.0))
res95_4: DenseVector[Double] = DenseVector(1.0, 4.0)
res95_5: DenseMatrix[Double] = 14.0  32.0  
32.0  77.0  

In [ ]:
## 応用: 簡単な統計分析
男女の体重と身長のデータをもつcsvファイルから、HWDataオブジェクトを生成する

In [3]:


import scala.reflect.ClassTag
import io.Source

import breeze.linalg._
import breeze.stats._
import breeze.optimize._

object HWData {
  
  val DataDirectory = "./"
  val fileName = "rep_height_weights.csv"
    
  def load:HWData =
  {
    val file = Source.fromFile(DataDirectory + fileName)
    val lines = file.getLines.toVector
    val splitLines = lines.map { _.split(',') }

    def fromList[T:ClassTag](index:Int, converter:(String => T)):DenseVector[T] =
      DenseVector.tabulate(lines.size) { irow => converter(splitLines(irow)(index)) }

    val genders = fromList(1, elem => elem.replace("\"", "").head)
    val weights = fromList(2, elem => elem.toDouble)
    val heights = fromList(3, elem => elem.toDouble)
    val reportedWeights = fromList(4, elem => elem.toDouble)
    val reportedHeights = fromList(5, elem => elem.toDouble)

    new HWData(weights, heights, reportedWeights, reportedHeights, genders)
  }

}

class HWData(
//     このようにそれぞれのデータに対応するベクトルをフィールドに持つように設計することで、Rなどと同じ記法でそれぞれのデータにアクセスできる
  val weights:DenseVector[Double],
  val heights:DenseVector[Double],
  val reportedWeights:DenseVector[Double],
  val reportedHeights:DenseVector[Double],
  val genders:DenseVector[Char]
) {

  val npoints = heights.length
  require(weights.length == npoints)
  require(reportedWeights.length == npoints)
  require(genders.length == npoints)
  require(reportedHeights.length == npoints)

  lazy val rescaledHeights:DenseVector[Double] =
    (heights - mean(heights)) / stddev(heights)

  lazy val rescaledWeights:DenseVector[Double] =
    (weights - mean(weights)) / stddev(weights)

  lazy val featureMatrix:DenseMatrix[Double] =
    DenseMatrix.horzcat( 
      DenseMatrix.ones[Double](npoints, 1), 
      rescaledHeights.toDenseMatrix.t,
      rescaledWeights.toDenseMatrix.t
    )

  lazy val target:DenseVector[Double] =
    genders.values.map { gender => if(gender == 'M') 1.0 else 0.0 }

  override def toString:String = s"HWData [ $npoints rows ]"

}

import scala.reflect.ClassTag
import io.Source
import breeze.linalg._
import breeze.stats._
import breeze.optimize._
defined object HWData
defined class HWData

# ロジスティック回帰



In [4]:
val data=HWData.load

data: HWData = HWData [ 181 rows ]

In [5]:
// データの正規化（正規化したデータは別メモリに保存される）
def rescale(v:DenseVector[Double])=
(v-mean(v))/stddev(v)

defined function rescale

In [6]:
data.heights

res5: DenseVector[Double] = DenseVector(182.0, 161.0, 161.0, 177.0, 157.0, 170.0, 167.0, 186.0, 178.0, 171.0, 175.0, 166.0, 161.0, 168.0, 163.0, 166.0, 187.0, 168.0, 197.0, 175.0, 180.0, 170.0, 175.0, 173.0, 171.0, 166.0, 169.0, 166.0, 157.0, 183.0, 166.0, 178.0, 173.0, 164.0, 169.0, 176.0, 166.0, 174.0, 178.0, 187.0, 164.0, 178.0, 163.0, 183.0, 179.0, 160.0, 174.0, 162.0, 182.0, 165.0, 169.0, 185.0, 176.0, 183.0, 172.0, 173.0, 165.0, 177.0, 180.0, 173.0, 189.0, 162.0, 165.0, 164.0, 158.0, 178.0, 175.0, 173.0, 165.0, 163.0, 166.0, 160.0, 160.0, 182.0, 183.0, 165.0, 168.0, 169.0, 167.0, 170.0, 182.0, 178.0, 165.0, 163.0, 162.0, 173.0, 161.0, 184.0, 180.0, 189.0, 165.0, 185.0, 169.0, 159.0, 164.0, 178.0, 163.0, 163.0, 175.0, 164.0, 152.0, 167.0, 166.0, 166.0, 183.0, 179.0, 174.0, 179.0, 167.0, 168.0, 184.0, 184.0, 169.0, 178.0, 178.0, 167.0, 178.0, 165.0, 157.0, 171.0, 157.0, 166.0, 185.0, 160.0, 148.0, 177.0, 162.0, 172.0, 188.0, 191.0, 175.0, 163.0, 165.0, 176.0, 171.0, 160.0, 165.0, 1

In [7]:
val rescaleHeights=rescale(data.heights)
val rescaleWeight=rescale(data.weights )

rescaleHeights: DenseVector[Double] = DenseVector(1.25556503726241, -1.0895959979387817, -1.0895959979387817, 0.6971933622145072, -1.536293337977104, -0.08452698285255666, -0.4195499878812983, 1.7022623773007324, 0.8088676972240878, 0.0271473521570239, 0.47384469219534614, -0.531224322890879, -1.0895959979387817, -0.3078756528717178, -0.8662473279196206, -0.531224322890879, 1.813936712310313, -0.3078756528717178, 2.9306800624061187, 0.47384469219534614, 1.032216367243249, -0.08452698285255666, 0.47384469219534614, 0.250496022176185, 0.0271473521570239, -0.531224322890879, -0.19620131786213724, -0.531224322890879, -1.536293337977104, 1.3672393722719907, -0.531224322890879, 0.8088676972240878, 0.250496022176185, -0.75457299291004, -0.19620131786213724, 0.5855190272049267, -0.531224322890879, 0.36217035718576557, 0.8088676972240878, 1.813936712310313, -0.75457299291004, 0.8088676972240878, -0.8662473279196206, 1.3672393722719907, 0.9205420322336684, -1.2012703329483623, 0.3621703571857655

In [8]:
// マトリックス化　181行1列
val rescaleHeightsAsMatrix=rescaleHeights.toDenseMatrix.t
val rescaleWeightsAsMatrix=rescaleWeight.toDenseMatrix.t

// 横方向に結合(一列目はバイアス(定数)項)
val featureMatrix=DenseMatrix.horzcat(
DenseMatrix.ones[Double](rescaleWeightsAsMatrix.rows,1),
rescaleHeightsAsMatrix ,
rescaleWeightsAsMatrix 
)



rescaleHeightsAsMatrix: DenseMatrix[Double] = 1.25556503726241      
-1.0895959979387817   
-1.0895959979387817   
0.6971933622145072    
-1.536293337977104    
-0.08452698285255666  
-0.4195499878812983   
1.7022623773007324    
0.8088676972240878    
0.0271473521570239    
0.47384469219534614   
-0.531224322890879    
-1.0895959979387817   
-0.3078756528717178   
-0.8662473279196206   
-0.531224322890879    
1.813936712310313     
-0.3078756528717178   
2.9306800624061187    
0.47384469219534614   
...
rescaleWeightsAsMatrix: DenseMatrix[Double] = 0.8415589308836935    
-0.5736592418835538   
-0.9460850768223031   
0.1711924279939448    
-0.49917407489580395  
0.7670737638959437    
0.7670737638959437    
0.24567759498169464   
0.39464792895719436   
-0.0522630729693048   
0.3201627619694445    
-0.6481444088713036   
-1.0950554107978028   
-0.12674823995705464  
-1.020570243810053    
-0.0522630729693048   
1.9588364356999415    
-0.2757185739325544   
0.7670737638959437    
-0.3502

In [9]:
// 識別用のターゲットデータの生成(M=1,F=0)
val target=data.genders.values.map{i=>
    if(i=='M') 1.0 else 0.0 
}


target: DenseVector[Double] = DenseVector(1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0)

In [14]:
// こうしてもよい　ただし、この場合maleVectors分のメモリを一時的に必要とする

val malesVecters=DenseVector.fill(data.genders.size)('M')
val target2=I(data.genders:==malesVecters)

malesVecters: DenseVector[Char] = DenseVector(M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M, M)
target2: DenseVector[Double] = DenseVector(1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0

In [10]:
import breeze.numerics._
import breeze.stats._
import breeze.optimize._

// 後でリファクタする
def costFunction(parameters:DenseVector[Double]):Double={
    val xBeta=featureMatrix * parameters
    val expXBeta=exp(xBeta)
    
//     最尤をlogをとったもの
    - sum((target :* xBeta)-log1p(expXBeta)) // log1p =log(1+x)
}

import breeze.numerics._
import breeze.stats._
import breeze.optimize._
defined function costFunction

In [11]:
// コスト関数を微分
def costFunctionGradient(parameters:DenseVector[Double]):DenseVector[Double]={
    val xBeta=featureMatrix * parameters
    val probs=sigmoid(xBeta)
    
    featureMatrix.t *(probs-target )
}

defined function costFunctionGradient

In [12]:
val f=new DiffFunction[DenseVector[Double]]{
   override def calculate(parameters:DenseVector[Double])=(costFunction(parameters),costFunctionGradient(parameters))
}

f: AnyRef with DiffFunction[DenseVector[Double]] = <function1>

In [13]:
// 最小化
val optiomalParameters=minimize(f,DenseVector(0.0,0.0,0.0))

7 29, 2017 9:11:35 午後 com.github.fommil.jni.JniLoader liberalLoad
情報: successfully loaded /var/folders/ng/kxk_y29n5xn0jyb_cr7_nc140000gq/T/jniloader1528702736597299004netlib-native_system-osx-x86_64.jnilib
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


optiomalParameters: DenseVector[Double] = DenseVector(-0.07514547434064464, 2.4762936471986414, 2.230545401350543)

## 上記のコードを再利用性が高いようにリファクタする

In [20]:
import scala.reflect.ClassTag
import io.Source

import breeze.linalg._
import breeze.stats._
import breeze.optimize._

class LogisticRegression(trainings:DenseMatrix[Double],targets:DenseVector[Double]){
    def costFunctionAndGradient(cof:DenseVector[Double]):(Double,DenseVector[Double])={
        val xBeta=trainings * cof
        val expXBeta=exp(xBeta)
        val cost= -sum((targets:*xBeta)-log1p(expXBeta))
        val probs=sigmoid(xBeta)
        val grad=trainings.t * (probs-targets)
        
        (cost,grad)
    }
    
    private def calculateOptimalCoefficient:DenseVector[Double]={
    val f=new DiffFunction[DenseVector[Double]]{
        def calculate(parameter:DenseVector[Double])=costFunctionAndGradient(parameter)
    }
        minimize(f,DenseVector.zeros[Double](trainings.cols ))
    }
    
    lazy val opticalCoeffcient=calculateOptimalCoefficient
}

import scala.reflect.ClassTag
import io.Source
import breeze.linalg._
import breeze.stats._
import breeze.optimize._
defined class LogisticRegression

In [21]:
val logistic=new LogisticRegression(featureMatrix ,target)

logistic: LogisticRegression = cmd19$$user$LogisticRegression@1d3a9683

In [22]:
logistic.opticalCoeffcient 

res21: DenseVector[Double] = DenseVector(-0.07514547434064464, 2.4762936471986414, 2.230545401350543)

# Scalaの並列処理の特性を活かして、交差検証を効率的に行う

In [ ]:
// CVFunction型を定義
type CVFunction = (Seq[Int], Seq[Int]) => Double

In [3]:
import breeze.linalg._
import breeze.numerics._

import breeze.linalg._
import breeze.numerics._

In [5]:
 val indexList = DenseVector.range(0, 10)
shuffle(indexList )

indexList: DenseVector[Int] = DenseVector(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
res4_1: DenseVector[Int] = DenseVector(0, 2, 6, 5, 9, 7, 3, 8, 4, 1)

In [11]:
// 交差検証のために、シャッフルして、トレーニングデータとテストデータにindexを分割するclassを作成する

// nEnumはデータの数、nCrossValidationsはテスト用に分割するデータの数
class RandamSample(nEnum:Int,nCrossValidations:Int){
//    f: 適用する学習ロジック (トレーニングデータのindex、テストデータのindex=>正答率 )
    type CVFunction = (Seq[Int],Seq[Int])=>Double
    
    require(nEnum>nCrossValidations,"与えるパラメータが違います")
    
    private  val indexList= DenseVector.range(0, nEnum)
//                            nShuffle: 交差検証を行う数。並列処理して行う事で、効率的に計算が可能
                       def mapSamples(nShuffle:Int)(f:CVFunction)={                           
//                            並列化
                           val cvResults= (0 to nShuffle).par.map{_=>
//                                shuffleで要素をシャッフル
                           val shuffleIndex=shuffle(indexList)
//                                split(元のベクトル,元のベクトルから最初の要素を取り出す数)
                            val Seq(testIndex,trainIndex)=  split(shuffleIndex,Seq(nCrossValidations))
                                   
                               // Vector型に変換し、学習ロジックを適用
                          f(testIndex.toScalaVector,trainIndex.toScalaVector)
                               
                          
                           }
DenseVector(cvResults.toArray) 
                       }
                       
                       
}

defined class RandamSample

## Akka Actorにより、クローニングシステムを構成する